## 3D Filters


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!ls '/content/drive/MyDrive/TFG 💪🧠/Code/DATA/'


P1  P2	P3  P4	P5  P6	P7  P8


In [3]:
!pip install SimpleITK
!pip install pynrrd
!pip install scikit-image
!pip install scipy
!pip install pandas
!pip install PyWavelets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 74.6 MB/s eta 0:00:00


In [4]:
import numpy as np
import scipy.ndimage as ndi
import SimpleITK as sitk
from scipy.ndimage import gaussian_filter, median_filter, laplace, sobel
import logging
from skimage import filters, exposure, morphology
from skimage.filters import frangi
from pathlib import Path
import os
from skimage.measure import regionprops
import logging
import nrrd
from skimage.measure import label, regionprops
from scipy.ndimage import gaussian_filter, median_filter, sobel
from pathlib import Path
import os
from skimage.measure import label, regionprops
from scipy.ndimage import gaussian_filter
import pandas as pd
from scipy.ndimage import median_filter
from skimage import restoration
from skimage.restoration import denoise_nl_means
import numpy as np
import scipy.ndimage as ndi
import matplotlib.pyplot as plt
from skimage.filters import threshold_otsu
from skimage.morphology import binary_closing, remove_small_objects
from scipy.ndimage import laplace
from skimage.filters import meijering
import pywt
from skimage.segmentation import felzenszwalb
from scipy.ndimage import grey_opening
from scipy.ndimage import distance_transform_edt, label
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from skimage.morphology import white_tophat, cube
from skimage.morphology import ball



# Set up logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')


In [ ]:
def read_nrrd_file(file_path):
    try:
        image = sitk.ReadImage(file_path)
        image_array = sitk.GetArrayFromImage(image)
        if len(image_array.shape) > 3:
            channel_dim = -1
            num_channels = image_array.shape[channel_dim]

            if num_channels in [3, 4]:
                logging.warning(f"⚠️ Image appears to be RGB/RGBA with {num_channels} channels. Converting to grayscale.")
                image_array = np.mean(image_array, axis=channel_dim)

        orig_min = np.min(image_array)
        orig_max = np.max(image_array)

        if orig_min != 0 or orig_max != 1:
            if orig_max > orig_min:
                image_array = (image_array - orig_min) / (orig_max - orig_min)
                logging.info(f"✅ Normalized image from range [{orig_min}, {orig_max}] to [0, 1]")
            else:
                logging.warning("⚠️ Image has constant value. Setting to zeros.")
                image_array = np.zeros_like(image_array)

        print(f"✅ Successfully read NRRD file: {file_path}")
        print(f"   Shape: {image_array.shape}, Range: [{np.min(image_array)}, {np.max(image_array)}]")

        return image_array, image

    except Exception as e:
        logging.error(f"❌ Error reading NRRD file: {str(e)}")
        return None, None

def save_nrrd_file(image_array, image, output_file):
    """Saves a processed volume as an NRRD file using SimpleITK."""
    try:

        if image_array.dtype == bool:
            image_array = image_array.astype(np.uint8)

        output_image = sitk.GetImageFromArray(image_array)
        output_image.SetSpacing(image.GetSpacing())
        output_image.SetOrigin(image.GetOrigin())
        output_image.SetDirection(image.GetDirection())

        sitk.WriteImage(output_image, output_file)
        print(f"✅ Saved NRRD file to {output_file}")

    except Exception as e:
        logging.error(f"❌ Error saving NRRD file: {str(e)}")

In [ ]:
def resample_to_isotropic(image, spacing=(1.0, 1.0, 1.0)):
    try:
        original_size = image.GetSize()
        original_spacing = image.GetSpacing()
        original_origin = image.GetOrigin()
        original_direction = image.GetDirection()

        print(f"Original size: {original_size}")
        print(f"Original spacing: {original_spacing}")
        print(f"Original origin: {original_origin}")
        print(f"Original direction: {original_direction}")
        print(f"Requested isotropic spacing: {spacing}")
        new_size = [
            int(round(s * o / ns))
            for s, o, ns in zip(original_size, original_spacing, spacing)
        ]

        print(f"Calculated new size: {new_size}")

        # Resample the image with the new size and isotropic spacing
        resampler = sitk.ResampleImageFilter()
        resampler.SetSize(new_size)
        resampler.SetOutputSpacing(spacing)
        resampler.SetOutputOrigin(original_origin)
        resampler.SetOutputDirection(original_direction)
        resampler.SetInterpolator(sitk.sitkLinear)

        resampled_image = resampler.Execute(image)

        print(f"Resampling successful. New size: {resampled_image.GetSize()}")
        return resampled_image

    except Exception as e:
        print(f"❌ Error in resampling: {e}")
        return None

def DoG(image, sigma1=1, sigma2=2):
    return gaussian_filter(image, sigma1) - gaussian_filter(image, sigma2)

def apply_brain_mask(image, brain_mask):
    return sitk.Mask(image, brain_mask)

def threshold_metal_voxels(image_array, percentile=99.5):
    threshold_value = np.percentile(image_array, percentile)
    return image_array > threshold_value

def apply_median_filter(image_array, kernel_size=3):
    return median_filter(image_array, size=kernel_size)

def remove_large_objects(volume, size_threshold):
    try:
        labeled_volume, num_features = label(volume > 0)
        regions = regionprops(labeled_volume)
        filtered_volume = np.zeros_like(volume)

        for region in regions:
            if region.area <= size_threshold:
                for coord in region.coords:
                    filtered_volume[tuple(coord)] = volume[tuple(coord)]

        print(f"✅ Removed {num_features - len([r for r in regions if r.area <= size_threshold])} large objects")
        return filtered_volume

    except Exception as e:
        logging.error(f"❌ Error removing large objects: {str(e)}")
        return volume


def vesselness_filter(image, sigma=1.0, alpha=0.5, beta=0.5):
    """
    Applies custom vesselness filter for enhancing vessels.

    Parameters:
        image (numpy.ndarray): The 3D image (volume) to be filtered.
        sigma (float): The standard deviation of the Gaussian filter used in Hessian computation.
        alpha (float): The sensitivity parameter for vesselness calculation.
        beta (float): The sensitivity parameter for vesselness calculation.

    Returns:
        numpy.ndarray: The vesselness-enhanced image.
    """
    print(f"Applying vesselness filter with sigma={sigma}, alpha={alpha}, beta={beta}")
    try:
        # Compute second derivatives
        Dxx = gaussian_filter(image, sigma=sigma, order=(2, 0, 0))
        Dyy = gaussian_filter(image, sigma=sigma, order=(0, 2, 0))
        Dzz = gaussian_filter(image, sigma=sigma, order=(0, 0, 2))
        Dxy = gaussian_filter(image, sigma=sigma, order=(1, 1, 0))
        Dxz = gaussian_filter(image, sigma=sigma, order=(1, 0, 1))
        Dyz = gaussian_filter(image, sigma=sigma, order=(0, 1, 1))

        # Initialize output
        vesselness = np.zeros_like(image)

        # Process each voxel
        for i in range(image.shape[0]):
            for j in range(image.shape[1]):
                for k in range(image.shape[2]):
                    # Construct Hessian matrix for this voxel
                    H = np.array([
                        [Dxx[i,j,k], Dxy[i,j,k], Dxz[i,j,k]],
                        [Dxy[i,j,k], Dyy[i,j,k], Dyz[i,j,k]],
                        [Dxz[i,j,k], Dyz[i,j,k], Dzz[i,j,k]]
                    ])

                    eigvals = np.linalg.eigvalsh(H)

                    # Sort eigenvalues by absolute value (|λ1| ≤ |λ2| ≤ |λ3|)
                    eigvals = np.sort(np.abs(eigvals))

                    # Compute vesselness - vessel structures have |λ1| ≈ 0, |λ2| ≈ |λ3| >> 0
                    if eigvals[1] > 0 and eigvals[2] > 0:
                        vessel_term = 1 - np.exp(-eigvals[1]**2 / (2 * alpha**2))
                        blob_term = np.exp(-eigvals[2]**2 / (2 * beta**2))
                        vesselness[i,j,k] = vessel_term * blob_term

        if vesselness.max() > vesselness.min():
            vesselness = (vesselness - vesselness.min()) / (vesselness.max() - vesselness.min())

        return vesselness

    except Exception as e:
        logging.error(f"Error applying vesselness filter: {str(e)}")
        return None

def sobel_filter(image):
    print("Applying Sobel edge detection filter")
    try:
        edges = np.sqrt(sobel(image, axis=0)**2 + sobel(image, axis=1)**2 + sobel(image, axis=2)**2)
        return edges
    except Exception as e:
        logging.error(f"Error applying Sobel filter: {str(e)}")
        return None


def gamma_correction(image, gamma=1.0):
    print(f"Applying Gamma correction with gamma={gamma}")
    try:
        image = np.power(image, gamma)
        return image
    except Exception as e:
        logging.error(f"Error applying gamma correction: {str(e)}")
        return None

def sharpen_high_pass(volume, strength=1):
    blurred = filters.gaussian(volume, sigma=1)
    sharpened = volume - blurred
    return volume + strength * sharpened

def hessian_filter(image, sigma=1.0):
    print(f"Applying Hessian filter with sigma={sigma}")
    try:
        Dxx = gaussian_filter(image, sigma=sigma, order=(2, 0, 0))
        Dyy = gaussian_filter(image, sigma=sigma, order=(0, 2, 0))
        Dzz = gaussian_filter(image, sigma=sigma, order=(0, 0, 2))
        Dxy = gaussian_filter(image, sigma=sigma, order=(1, 1, 0))
        Dxz = gaussian_filter(image, sigma=sigma, order=(1, 0, 1))
        Dyz = gaussian_filter(image, sigma=sigma, order=(0, 1, 1))

        result = np.zeros_like(image)

        for i in range(image.shape[0]):
            for j in range(image.shape[1]):
                for k in range(image.shape[2]):

                    H = np.array([
                        [Dxx[i,j,k], Dxy[i,j,k], Dxz[i,j,k]],
                        [Dxy[i,j,k], Dyy[i,j,k], Dyz[i,j,k]],
                        [Dxz[i,j,k], Dyz[i,j,k], Dzz[i,j,k]]
                    ])

                    eigvals = np.linalg.eigvalsh(H)
                    result[i,j,k] = np.prod(eigvals)

        return result

    except Exception as e:
        logging.error(f"Error applying Hessian filter: {str(e)}")
        return None


def anisotropic_diffusion(image, num_iter=10, kappa=50, gamma=0.1):
    """Apply Anisotropic Diffusion (Perona-Malik) on a 3D image (volume)."""
    image = np.asarray(image, dtype=np.float64)

    print("Input image shape before anisotropic diffusion:", image.shape)

    for iteration in range(num_iter):
        print(f"Iteration {iteration + 1}/{num_iter}")

        diff_x = np.zeros_like(image)
        diff_x[:, :, 1:] = image[:, :, 1:] - image[:, :, :-1]

        # For Y axis (axis=1)
        diff_y = np.zeros_like(image)
        diff_y[:, 1:, :] = image[:, 1:, :] - image[:, :-1, :]

        # For Z axis (axis=0)
        diff_z = np.zeros_like(image)
        diff_z[1:, :, :] = image[1:, :, :] - image[:-1, :, :]

        # Compute diffusion coefficients
        c_x = np.exp(-(diff_x / kappa) ** 2)
        c_y = np.exp(-(diff_y / kappa) ** 2)
        c_z = np.exp(-(diff_z / kappa) ** 2)

        # Compute flux terms
        flux_x = c_x * diff_x
        flux_y = c_y * diff_y
        flux_z = c_z * diff_z

        # Compute divergence (update step)
        div_x = np.zeros_like(image)
        div_x[:, :, :-1] += flux_x[:, :, :-1]
        div_x[:, :, 1:] -= flux_x[:, :, :-1]

        div_y = np.zeros_like(image)
        div_y[:, :-1, :] += flux_y[:, :-1, :]
        div_y[:, 1:, :] -= flux_y[:, :-1, :]

        div_z = np.zeros_like(image)
        div_z[:-1, :, :] += flux_z[:-1, :, :]
        div_z[1:, :, :] -= flux_z[:-1, :, :]

        # Update image
        image += gamma * (div_x + div_y + div_z)

        print(f"Image shape after iteration {iteration + 1}: {image.shape}")
        print(f"Min value after iteration {iteration + 1}: {np.min(image)}")
        print(f"Max value after iteration {iteration + 1}: {np.max(image)}")

    return image

def denoise_3d(volume, patch_size = 3, patch_distance= 3, h=0.3):
    return denoise_nl_means(volume, patch_size, patch_distance,h)


def threshold_metal_voxels_3D(image_array, percentile=99.5):
    """Threshold metal voxels across the entire 3D volume."""
    print(f"DEBUG: Input array shape: {image_array.shape}, dtype: {image_array.dtype}")

    if image_array.ndim != 3:
        raise ValueError("Input must be a 3D array.")

    # Flatten the 3D array and compute the percentile value over the entire volume
    flattened_data = image_array.flatten()
    print(f"DEBUG: Flattened data shape: {flattened_data.shape}, dtype: {flattened_data.dtype}")

    # Remove NaN and Inf values from the flattened data
    finite_data = flattened_data[np.isfinite(flattened_data)]
    print(f"DEBUG: Finite data shape: {finite_data.shape}, dtype: {finite_data.dtype}")

    # Compute the threshold over the entire 3D volume
    threshold_value = np.percentile(finite_data, percentile)
    print(f"DEBUG: Threshold value: {threshold_value}")

    # Apply the threshold across the entire 3D array
    mask = (image_array > threshold_value).astype(np.uint8)
    print(f"DEBUG: Final mask shape: {mask.shape}, dtype: {mask.dtype}")
    print(f"DEBUG: Unique values in mask: {np.unique(mask)}")

    return mask

def apply_wavelets_3d(volume, wavelet='db2', threshold=0.2):
    # Perform 3D wavelet decomposition (using discrete wavelet transform)
    coeffs = pywt.wavedec3(volume, wavelet, level=3)

    # Apply thresholding to the wavelet coefficients to reduce noise
    coeffs = list(coeffs)
    coeffs[1:] = [(pywt.threshold(cH, threshold, mode='soft'),
                   pywt.threshold(cV, threshold, mode='soft'),
                   pywt.threshold(cD, threshold, mode='soft'))
                  for (cH, cV, cD) in coeffs[1:]]
    denoised_volume = pywt.waverec3(coeffs, wavelet)
    denoised_volume = np.clip(denoised_volume, 0, 255)

    return denoised_volume


def snake_segmentation_3D(volume, alpha=0.01, beta=0.1, gamma=0.01):
    """
    Applies active contour segmentation (Snake Algorithm) slice by slice.
    """
    segmented_slices = np.zeros_like(volume)

    for i in range(volume.shape[0]):
        slice_image = volume[i]
        s = np.linspace(0, 2*np.pi, 400)
        x = 128 + 60*np.cos(s)
        y = 128 + 60*np.sin(s)
        init = np.array([x, y]).T

        snake = active_contour(slice_image, init, alpha=alpha, beta=beta, gamma=gamma)

        mask = np.zeros_like(slice_image)
        for x, y in snake:
            mask[int(y), int(x)] = 255

        segmented_slices[i] = mask

    return segmented_slices

def gaussian_pyramid(volume, max_level=3):
    pyramid = [volume]
    for i in range(max_level):
        volume = cv2.pyrDown(volume)  # Reduce image size and apply Gaussian blur
        pyramid.append(volume)
    return pyramid

def laplacian_pyramid(volume, max_level=3):
    gaussian_pyr = gaussian_pyramid(volume, max_level)
    laplacian_pyr = []

    for i in range(max_level):
        gaussian_expanded = cv2.pyrUp(gaussian_pyr[i + 1])
        laplacian = cv2.subtract(gaussian_pyr[i], gaussian_expanded)
        laplacian_pyr.append(laplacian)

    laplacian_pyr.append(gaussian_pyr[-1])
    return laplacian_pyr


def remove_outliers_iqr(mask):
    """
    Removes outliers from the mask using the IQR method.

    Args:
        mask (ndarray): The binary mask or feature array (e.g., centroid, volume).

    Returns:
        ndarray: Mask with outliers removed.
    """
    # Flatten the mask
    mask_values = mask.flatten()

    # Compute the first and third quartiles (Q1, Q3)
    Q1 = np.percentile(mask_values, 25)
    Q3 = np.percentile(mask_values, 75)

    # Calculate IQR
    IQR = Q3 - Q1

    # Define the bounds for outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Identify and remove outliers
    mask[(mask_values < lower_bound) | (mask_values > upper_bound)] = 0

    return mask

def remove_outliers_zscore(mask, threshold=3):
    """
    Removes outliers from the mask using Z-score thresholding.

    Args:
        mask (ndarray): The binary mask or feature array (e.g., centroid, volume).
        threshold (float): The Z-score threshold to classify outliers.

    Returns:
        ndarray: Mask with outliers removed.
    """
    mask_values = mask.flatten()
    z_scores = zscore(mask_values)
    outliers = np.abs(z_scores) > threshold
    mask[outliers] = 0

    return mask

def grayscale_opening_3d(volume, spacing, min_dist_mm=0.2):
    """
    Applies a 3D grayscale opening operation (erosion followed by dilation)
    to enhance structures in the volume based on a minimum distance.

    Args:
        volume (numpy array): 3D volume data (Z, Y, X).
        spacing (tuple): Voxel spacing in (z, y, x) directions (mm).
        min_dist_mm (float): Minimum distance in mm to determine kernel size.

    Returns:
        numpy array: Volume with enhanced structures after opening.
    """
    kernel_radius = min_dist_mm / spacing[0]
    voxel_kernel_size = max(1, int(round(kernel_radius)))
    structuring_element = ball(voxel_kernel_size)
    opened_volume = grey_opening(volume, footprint=structuring_element)

    return opened_volume


def top_hat_3d(volume, structure_size=1):
    """
    Applies a 3D white top-hat transformation to enhance small bright objects.

    Args:
        volume (numpy array): 3D volume data (Z, Y, X)
        structure_size (int): Size of the structuring element (cube).

    Returns:
        numpy array: Volume with enhanced bright structures.
    """
    # Create a cubic structuring element (3D ball)
    structure = np.ones((structure_size, structure_size, structure_size))

    # Apply grayscale opening to the 3D volume
    opened_volume = grey_opening(volume, footprint=structure)

    # Top-hat result: Original volume - opened volume
    tophat_result = volume - opened_volume

    return tophat_result

def separate_merged_3d(mask, electrode_radius=0.4, voxel_size=(1, 1, 1), distance_threshold=1):
    mask = mask.astype(bool)
    distance = distance_transform_edt(mask)
    thresholded_distance = distance > distance_threshold
    labeled_mask, num_features = label(thresholded_distance)
    separated_mask = np.zeros_like(mask)
    for i in range(1, num_features + 1):
        separated_mask[labeled_mask == i] = 1

    return separated_mask

def rescale_volume(volume):
    min_value = np.min(volume)
    max_value = np.max(volume)

    if max_value == min_value:
        return np.zeros_like(volume)

    return (volume - min_value) / (max_value - min_value)

def apply_threshold(volume, original_threshold):
    min_value, max_value = np.min(volume), np.max(volume)
    normalized_volume = rescale_volume(volume)

    normalized_threshold = (original_threshold - min_value) / (max_value - min_value)

    return np.uint8(normalized_volume > normalized_threshold) * 25

# Process volume with cascading filters using CPU

In [ ]:
def process_volume_3d(input_volume_path, input_ctp_path = None, output_dir=None, filename_pattern="Enhanced_3D_filters_{name}_{original}"):
    """Processes the volume with multiple filters in sequence."""
    try:
        # Load the input volume
        image_array, image = read_nrrd_file(input_volume_path)
        if input_ctp_path is not None:
            ctp_array, ctp = read_nrrd_file(input_ctp_path)

        if image_array is None or image is None:
            print("❌ Failed to load input volume.")
            return

        # Apply filters in sequence
        enhanced_volumes = {}

        ###### CTP without mask ####
        enhanced_volumes['CTP_volume_gauss'] = filters.gaussian(ctp_array, sigma=0.5, preserve_range=True)
        enhanced_volumes['CTP_volume_gamma'] = gamma_correction(enhanced_volumes['CTP_volume_gauss'], gamma = 4)
        enhanced_volumes['CTP_volume_chambolle'] = restoration.denoise_tv_chambolle(enhanced_volumes['CTP_volume_gamma'], weight = 0.03)
        enhanced_volumes['CTP_volume_gauss_2'] = filters.gaussian(enhanced_volumes['CTP_volume_chambolle'], sigma=0.2, preserve_range=True)
        enhanced_volumes['CTP_volume_tophat'] = top_hat_3d(enhanced_volumes['CTP_volume_chambolle'])
        enhanced_volumes['CTP_volume_high_pass'] = sharpen_high_pass(enhanced_volumes['CTP_volume_tophat'], strength=0.3)
        enhanced_volumes['CTP_volume_gamma'] = gamma_correction(enhanced_volumes['CTP_volume_high_pass'], gamma = 3)
        enhanced_volumes['CTP_volume_threshold'] = threshold_metal_voxels_3D(enhanced_volumes['CTP_volume_gamma'] , percentile=99.9)


        # Apply filters in sequence 'normal'
        enhanced_volumes['ROI_volume_chambolle'] = restoration.denoise_tv_chambolle(image_array, weight = 0.03)
        enhanced_volumes['ROI_Threshold_chambolle'] = np.uint8(enhanced_volumes['ROI_volume_chambolle'] > 0.463)
        enhanced_volumes['ROI_volume_DoG'] = DoG(image_array, sigma1=0.5, sigma2=1)
        enhanced_volumes['ROI_Threshold_DoG'] = np.uint8(enhanced_volumes['ROI_volume_DoG'] > 0.128)
        enhanced_volumes['ROI_gamma_ROI'] = gamma_correction(enhanced_volumes['ROI_volume_DoG'], gamma = 3)
        enhanced_volumes['ROI_Threshold_GAMMA_ROI'] = np.uint8(enhanced_volumes['ROI_gamma_ROI'] > 0.0019)
        enhanced_volumes['ROI_gamma'] = gamma_correction(enhanced_volumes['ROI_volume_DoG'], gamma = 3)
        enhanced_volumes['ROI_sharpen_high_pass'] = sharpen_high_pass(enhanced_volumes['ROI_gamma'], strength=0.3)
        enhanced_volumes['ROI_THRESHOLD_HIGH'] = np.uint8(enhanced_volumes['ROI_sharpen_high_pass'] > 0.0022)
        enhanced_volumes['ROI_thresholding_percentile'] = threshold_metal_voxels_3D(enhanced_volumes['ROI_sharpen_high_pass'], percentile=99.8)

        ####simple####

        enhanced_volumes['SIMPLE_gaussian'] = filters.gaussian(image_array, sigma=0.3, preserve_range=True)
        enhanced_volumes['SIMPLE_FINAL_THRESHOLD'] = np.uint8(enhanced_volumes['SIMPLE_gaussian'] > 0.419)
        enhanced_volumes['SIMPLE_NOT_WATERSHED'] = top_hat_3d(enhanced_volumes['SIMPLE_FINAL_THRESHOLD'])
        enhanced_volumes['SIMPLE_GAMMA'] = gamma_correction(enhanced_volumes['SIMPLE_gaussian'], gamma = 3)
        enhanced_volumes['SIMPLE_sharpened'] = sharpen_high_pass(enhanced_volumes['SIMPLE_GAMMA'], strength=0.5)
        enhanced_volumes['SIMPLE_FINAL_SHARPENED'] = apply_threshold(rescale_volume(enhanced_volumes['SIMPLE_sharpened']), 0.14)
        enhanced_volumes['SIMPLE_opening'] = grayscale_opening_3d(enhanced_volumes['SIMPLE_sharpened'], spacing=(1,1,1), min_dist_mm=0.5)
        enhanced_volumes['SIMPLE_sheperned_2'] = sharpen_high_pass(enhanced_volumes['SIMPLE_opening'], strength=0.3)
        enhanced_volumes['SIMPLE_gamma_2'] = gamma_correction(enhanced_volumes['SIMPLE_sheperned_2'], gamma = 3)
        enhanced_volumes['SIMPLE_FINAL_gamma_2'] = np.uint8(enhanced_volumes['SIMPLE_gamma_2'] > 0.02)
        enhanced_volumes['SIMPLE_thresholding_percentile'] = threshold_metal_voxels_3D(enhanced_volumes['SIMPLE_gamma_2'], percentile=99.9)

        ####simple####

        if output_dir:
            output_path = Path(output_dir)
            output_path.mkdir(parents=True, exist_ok=True)

            for name, volume in enhanced_volumes.items():
                filename = filename_pattern.format(name=name, original=os.path.basename(input_volume_path))
                output_file = output_path / filename
                save_nrrd_file(volume, image, str(output_file))

        return enhanced_volumes

    except Exception as e:
        logging.error(f"Error during volume processing: {str(e)}")

In [ ]:
from pathlib import Path

output_dir = '/content/drive/MyDrive/TFG 💪🧠/Code/DATA/P1/P1/Enhance_ctp_3D'
output_path = Path(output_dir)
output_path.mkdir(parents=True, exist_ok=True)
if output_path.exists():
    print(f"✅ Output directory exists or was created: {output_dir}")
else:
    print(f"❌ Failed to create output directory: {output_dir}")


✅ Output directory exists or was created: /content/drive/MyDrive/TFG 💪🧠/Code/DATA/P1/P1/Enhance_ctp_3D


In [ ]:
# Example
input_volume_path ='/content/drive/MyDrive/TFG 💪🧠/Code/DATA/P1/P1/Filtered_th_20_PRUEBA_roi_plus_gamma_mask_ctp.3D.nrrd'
input_ctp_path = '/content/drive/MyDrive/TFG 💪🧠/Code/DATA/P1/P1/Filtered_volume_array_ctp.3D.nrrd'

process_volume_3d(input_volume_path,input_ctp_path ,output_dir=output_dir,filename_pattern='Filtered_3D_{name}_{original}')

✅ Successfully read NRRD file: /content/drive/MyDrive/TFG 💪🧠/Code/DATA/P1/P1/Filtered_th_20_PRUEBA_roi_plus_gamma_mask_ctp.3D.nrrd
   Shape: (256, 256, 256), Range: [0.0, 1.0]
✅ Successfully read NRRD file: /content/drive/MyDrive/TFG 💪🧠/Code/DATA/P1/P1/Filtered_volume_array_ctp.3D.nrrd
   Shape: (256, 256, 256), Range: [0.0, 1.0]
Applying Gamma correction with gamma=4
Applying Gamma correction with gamma=3
DEBUG: Input array shape: (256, 256, 256), dtype: float32
DEBUG: Flattened data shape: (16777216,), dtype: float32
DEBUG: Finite data shape: (16777216,), dtype: float32
DEBUG: Threshold value: 0.0
DEBUG: Final mask shape: (256, 256, 256), dtype: uint8
DEBUG: Unique values in mask: [0]
Applying Gamma correction with gamma=3
Applying Gamma correction with gamma=3
DEBUG: Input array shape: (256, 256, 256), dtype: float64
DEBUG: Flattened data shape: (16777216,), dtype: float64
DEBUG: Finite data shape: (16777216,), dtype: float64
DEBUG: Threshold value: 0.00026301109906717526
DEBUG: Fin

{'CTP_volume_gauss': array([[[0.49614438, 0.49614438, 0.49614438, ..., 0.49614438,
          0.49614438, 0.49614438],
         [0.49614438, 0.49614438, 0.49614438, ..., 0.49614438,
          0.49614438, 0.49614438],
         [0.49614438, 0.49614438, 0.49614438, ..., 0.49614438,
          0.49614438, 0.49614438],
         ...,
         [0.49614438, 0.49614438, 0.49614438, ..., 0.49614438,
          0.49614438, 0.49614438],
         [0.49614438, 0.49614438, 0.49614438, ..., 0.49614438,
          0.49614438, 0.49614438],
         [0.49614438, 0.49614438, 0.49614438, ..., 0.49614438,
          0.49614438, 0.49614438]],
 
        [[0.49614438, 0.49614438, 0.49614438, ..., 0.49614438,
          0.49614438, 0.49614438],
         [0.49614438, 0.49614438, 0.49614438, ..., 0.49614438,
          0.49614438, 0.49614438],
         [0.49614438, 0.49614438, 0.49614438, ..., 0.49614438,
          0.49614438, 0.49614438],
         ...,
         [0.49614438, 0.49614438, 0.49614438, ..., 0.49614438,
    

Labeling and centroides



In [ ]:
def detect_shape_and_coordinates(volume, shape_type='sphere', threshold=0.5):
    """
    Detects regions in a 3D volume based on their shape and returns their coordinates and centroids.
    """
    try:
        # Step 1: Apply threshold to the volume
        binary_volume = volume > threshold

        # Step 2: Label the connected components in the binary volume
        labeled_volume, num_labels = label(binary_volume)

        # Step 3: Analyze each region (connected component)
        shapes_data = []

        for region in regionprops(labeled_volume):
            # Calculate the centroid of the region
            centroid = region.centroid

            # Get the coordinates of the region
            region_coords = region.coords

            # Calculate the bounding box and aspect ratio (for shape detection)
            minr, minc, minz, maxr, maxc, maxz = region.bbox
            length = maxr - minr
            width = maxc - minc
            height = maxz - minz

            # Aspect ratio for shape classification (simple criteria for shapes)
            aspect_ratio = max(length, width, height) / min(length, width, height)

            # Check shape types based on aspect ratio and volume
            if shape_type == 'sphere':
                # For spheres, aspect ratio should be close to 1
                if 0.9 <= aspect_ratio <= 1.1:
                    shapes_data.append({
                        'type': 'sphere',
                        'centroid': centroid,
                        'coordinates': region_coords.tolist(),
                        'volume': region.area
                    })
            elif shape_type == 'square':
                # For squares (in 2D) or cubes in 3D, aspect ratio should be close to 1
                if 0.9 <= aspect_ratio <= 1.1 and region.area > 100:  # Only considering large regions
                    shapes_data.append({
                        'type': 'square',
                        'centroid': centroid,
                        'coordinates': region_coords.tolist(),
                        'volume': region.area
                    })
            elif shape_type == 'rectangle':
                # For rectangles (or rectangular prisms in 3D), aspect ratio should be greater than 1
                if aspect_ratio > 1.2:
                    shapes_data.append({
                        'type': 'rectangle',
                        'centroid': centroid,
                        'coordinates': region_coords.tolist(),
                        'volume': region.area
                    })

        # Step 4: Convert results into a DataFrame for easy export
        shapes_df = pd.DataFrame(shapes_data)
        return shapes_df

    except Exception as e:
        logging.error(f"Error detecting shapes: {str(e)}")
        return None


# Example usage
volume = np.random.random((100, 100, 100))  # Replace with your 3D volume (e.g., from NRRD file)
shapes_df = detect_shape_and_coordinates(volume, shape_type='sphere', threshold=0.5)

if shapes_df is not None:
    # Save to CSV or print the results
    shapes_df.to_csv('shapes_data.csv', index=False)
    print(shapes_df)

        type                                           centroid  \
0     sphere  (49.508062239424575, 49.46617991825245, 49.488...   
1     sphere                                   (0.0, 0.0, 26.0)   
2     sphere                                   (0.0, 0.0, 57.0)   
3     sphere                                   (0.4, 0.4, 62.8)   
4     sphere                                   (0.0, 0.0, 91.0)   
...      ...                                                ...   
8375  sphere                                 (99.0, 98.0, 57.0)   
8376  sphere                                 (99.0, 98.0, 63.0)   
8377  sphere                                 (99.0, 99.0, 17.0)   
8378  sphere                                 (99.0, 99.0, 22.0)   
8379  sphere                                 (99.0, 99.0, 56.0)   

                                            coordinates    volume  
0     [[0, 0, 0], [0, 0, 1], [0, 0, 6], [0, 0, 7], [...  488822.0  
1                                          [[0, 0, 26]]    